<a href="https://colab.research.google.com/github/ElsChar31/Amazon_Vine_Analysis/blob/main/Amazon_ETL_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1  '
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
Execution output
3KB
	Stream
		0% [Working]
            
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
		
0% [Connecting to archive.ubuntu.com (91.189.88.152)] [1 InRelease 0 B/88.7 kB 
0% [Waiting for headers] [Connected to cloud.r-project.org (65.8.183.17)] [Wait
                                                                               
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
		Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
		Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
		Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
		Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
		Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
		Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
		Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
		Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
		Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [31.5 kB]
		Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
		Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,420 kB]
		Get:14 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [500 kB]
		Get:15 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,257 kB]
		Get:16 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
		Hit:17 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
		Get:18 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
		Ign:20 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
		Get:20 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [679 kB]
		Get:21 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,785 kB]
		Get:22 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [533 kB]
		Get:23 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [38.7 kB]
		Get:24 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [2,693 kB]
		Get:25 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [913 kB]
		Get:26 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,190 kB]
		Get:27 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [44.1 kB]
		Fetched 13.4 MB in 4s (3,283 kB/s)
		Reading package lists... Done

Code cell <_xKwTpATHqSe>
#%% [code]
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Execution output
1KB
	Stream
		--2021-07-22 19:56:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
		Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
		Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
		HTTP request sent, awaiting response... 200 OK
		Length: 1002883 (979K) [application/java-archive]
		Saving to: ‘postgresql-42.2.16.jar’
		
		postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    
		
		2021-07-22 19:56:15 (6.59 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]

Code cell <MMqDAjVS0KN9>
#%% [code]
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Text cell <cyBsySGuY-9V>
#%% [markdown]
### Load Amazon Data into Spark DataFrame

Code cell <CtCmBhQJY-9Z>
#%% [code]
from pyspark import SparkFiles
url = ""
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

Text cell <2yUSe55VY-9t>
#%% [markdown]
### Create DataFrames to match tables

Code cell <C8REmY1aY-9u>
#%% [code]
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


Code cell <B0TESUDRY-90>
#%% [code]
# Create the customers_table DataFrame
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")

Code cell <4FwXA6UvY-96>
#%% [code]
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()

Code cell <MkqyCuNQY-9->
#%% [code]
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

Code cell <lzMmkdKmY--D>
#%% [code]
# Create the vine_table. DataFrame
# vine_df = df.select([])

Text cell <jITZhLkmY--J>
#%% [markdown]
### Connect to the AWS RDS instance and write each DataFrame to its table. 

Code cell <7jiUvs1aY--L>
#%% [code]
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/<database name>"
config = {"user":"postgres", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

Code cell <T2zgZ-aKY--Q>
#%% [code]
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Code cell <1m3yzn-LY--U>
#%% [code]
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Code cell <KbXri15fY--Z>
#%% [code]
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

Code cell <XdQknSHLY--e>
#%% [code]
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Code cell <Exuo6ebUsCqW>
#%% [code]